In [2]:
import numpy as np
import pandas as pd

# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [70]:
df = pd.read_csv('data/adult.csv')
df = df.replace("?", pd.NA).dropna()
df["salary"].value_counts()
df["salary K$"].value_counts()

25     687
46     681
38     681
16     678
22     677
      ... 
153     15
183     15
94      15
294     14
207     13
Name: salary K$, Length: 334, dtype: int64

# Task 1
Print the count of men and women in the dataset.

In [69]:
men = df['sex'].value_counts().get('Male')
women = df['sex'].value_counts().get('Female')

# Task 2
Find the average age of men in dataset

In [75]:
df[df['sex'] == 'Male']['age'].mean()

39.18400392541707

# Task 3
Get the percentage of people from Poland (native-country)

In [76]:
df[df['native-country'] == 'Poland'].shape[0] / df.shape[0] * 100

0.18566408063125786

# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [79]:
df.head()
stats1 = df[df['salary'] == '>50K']['age'].agg(['mean', 'std'])
stats2 = df[df['salary'] == '<= 50K']['age'].agg(['mean', 'std'])

# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [84]:
high_income_no_higher_edu = df[(~df['education'].isin(['Bachelors', 'Prof-school', 'Assoc-acdm', 'Assoc-voc', 'Masters', 'Doctorate'])) & (df['salary'] == '>50K')]

# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [85]:
education_stats = df.groupby('education')['age'].describe()

# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [90]:
male_df = df[df['sex'] == 'Male']
salary_comparison = male_df.groupby([df['marital-status'].str.startswith('Married'), 'salary'])['salary'].count().unstack()

# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [89]:
max_hours_per_week = df['hours-per-week'].max()
people_with_max_hours = df[df['hours-per-week'] == max_hours_per_week].shape[0]

# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight thier connection.

In [91]:
df.corr()

,Unnamed: 0,age,hours-per-week,salary K$
Unnamed: 0,1.000000,-0.001126,-0.001890,0.000129
age,-0.001126,1.000000,0.101599,0.208203
hours-per-week,-0.001890,0.101599,1.000000,0.196378
salary K$,0.000129,0.208203,0.196378,1.000000
